<a href="https://colab.research.google.com/github/samina-if/AdventOfCode2024/blob/main/Advent_of_Code_Day14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Part 1 Day 14
def parse_input(file_path):
    """Parse the input file to extract positions and velocities."""
    robots = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            p = tuple(map(int, parts[0][2:].split(',')))
            v = tuple(map(int, parts[1][2:].split(',')))
            robots.append((p, v))
    return robots

def simulate_robots(robots, width, height, seconds):
    """Simulate robot motion with wrapping for a given number of seconds."""
    positions = {}

    for p, v in robots:
        # Calculate new position after `seconds` seconds
        x = (p[0] + v[0] * seconds) % width
        y = (p[1] + v[1] * seconds) % height
        new_pos = (x, y)

        # Count robots in each position
        if new_pos not in positions:
            positions[new_pos] = 0
        positions[new_pos] += 1

    return positions

def compute_safety_factor(positions, width, height):
    """Compute the safety factor based on robot counts in quadrants."""
    mid_x = width // 2
    mid_y = height // 2

    quadrants = [0, 0, 0, 0]  # Q1, Q2, Q3, Q4

    for (x, y), count in positions.items():
        if x == mid_x or y == mid_y:
            continue  # Ignore robots on the middle lines

        if x < mid_x and y < mid_y:
            quadrants[1] += count  # Q2
        elif x >= mid_x and y < mid_y:
            quadrants[0] += count  # Q1
        elif x < mid_x and y >= mid_y:
            quadrants[2] += count  # Q3
        elif x >= mid_x and y >= mid_y:
            quadrants[3] += count  # Q4

    # Multiply the counts of robots in the quadrants
    safety_factor = quadrants[0] * quadrants[1] * quadrants[2] * quadrants[3]
    return safety_factor

if __name__ == "__main__":
    # Constants for the problem
    WIDTH = 101
    HEIGHT = 103
    SECONDS = 100

    # Read input
    input_file = "input.txt"
    robots = parse_input(input_file)

    # Simulate motion
    positions = simulate_robots(robots, WIDTH, HEIGHT, SECONDS)

    # Compute and print the safety factor
    safety_factor = compute_safety_factor(positions, WIDTH, HEIGHT)
    print("Safety Factor:", safety_factor)

In [ ]:
# Part 2
import numpy as np
from typing import List, Tuple

def parse_input(filename: str) -> List[Tuple[Tuple[int, int], Tuple[int, int]]]:
    """
    Parse input file to extract robot positions and velocities.

    Args:
        filename (str): Path to input file

    Returns:
        List of tuples, each containing (position, velocity)
    """
    robots = []
    with open(filename, 'r') as f:
        for line in f:
            pos_str, vel_str = line.strip().split(' ')
            px, py = map(int, pos_str[2:].split(','))
            vx, vy = map(int, vel_str[2:].split(','))
            robots.append(((px, py), (vx, vy)))
    return robots

def simulate_robots(robots: List[Tuple[Tuple[int, int], Tuple[int, int]]],
                    time: int,
                    width: int,
                    height: int) -> np.ndarray:
    """
    Simulate robot movements and count robot positions after given time.

    Args:
        robots (List): List of robot (position, velocity) tuples
        time (int): Seconds to simulate
        width (int): Space width
        height (int): Space height

    Returns:
        numpy array of robot count per tile
    """
    # Initialize grid to track robot positions
    grid = np.zeros((height, width), dtype=int)

    for (px, py), (vx, vy) in robots:
        # Calculate final position after time, with wraparound
        final_x = (px + vx * time) % width
        final_y = (py + vy * time) % height
        grid[final_y, final_x] += 1

    return grid

def is_christmas_tree(grid: np.ndarray) -> bool:
    """
    Check if the robot positions form a Christmas tree pattern.

    Args:
        grid (numpy array): Robot position grid

    Returns:
        Boolean indicating if a Christmas tree pattern is found
    """
    # Define a potential Christmas tree pattern
    # This is a simplistic representation and might need adjustment
    # based on the actual expected pattern
    tree_pattern = [
        [0, 0, 1, 0, 0],   # top of tree
        [0, 1, 1, 1, 0],   # middle of tree
        [1, 1, 1, 1, 1],   # base of tree
    ]

    height, width = grid.shape

    # Search for the pattern in the grid
    for y in range(height - len(tree_pattern)):
        for x in range(width - len(tree_pattern[0])):
            # Check if this section matches the tree pattern
            match = True
            for dy, row in enumerate(tree_pattern):
                for dx, val in enumerate(row):
                    if val == 1 and grid[y+dy, x+dx] == 0:
                        match = False
                        break
                if not match:
                    break

            if match:
                return True

    return False

def find_christmas_tree_time(filename: str, max_time: int = 10000, width: int = 101, height: int = 103) -> int:
    """
    Find the earliest time when robots form a Christmas tree pattern.

    Args:
        filename (str): Input file path
        max_time (int): Maximum time to search
        width (int): Space width
        height (int): Space height

    Returns:
        Time when Christmas tree pattern is found, or -1 if not found
    """
    # Parse input
    robots = parse_input(filename)

    # Search for Christmas tree pattern
    for time in range(max_time):
        # Simulate robot positions
        grid = simulate_robots(robots, time, width, height)

        # Check for Christmas tree pattern
        if is_christmas_tree(grid):
            return time

    return -1  # No pattern found within max_time

# Solve problem
result = find_christmas_tree_time('input.txt')
print(f"Seconds until Christmas Tree Pattern: {result}")